## Experiment 4
We want to evaluate the effect of converting string to unique ID respect to the Ri values

### First train the model with new encoding

In [1]:
import deepmatcher as dm
from utilities_functions.intermediate_layer_extraction import Hook,return_layer_input_output
import torch
from deepmatcher.data import MatchingIterator

torch.cuda.is_available()

True

In [3]:
hybrid_model_numericencoding = dm.MatchingModel(attr_summarizer='hybrid')
train,valid,test = dm.data.process('sample_data/itunes-amazon/exp4',train='train_numericencoding.csv',
                                  validation='validation_numericencoding.csv',test='test_numericencoding.csv')

In [4]:
hybrid_model_numericencoding.load_state('models/hybrid_numericencoding.pth')
hybrid_model_numericencoding.cuda()

MatchingModel(
  (attr_summarizers): ModuleMap(
    (Song_Name): Hybrid(
      (word_contextualizer): RNN(
        (rnn_groups): ModuleList(
          (0): GRU(300, 150, batch_first=True, bidirectional=True)
        )
        (dropouts): ModuleList(
          (0): Dropout(p=0)
        )
        (bypass_networks): ModuleList(
          (0): None
        )
        (input_dropout): NoMeta(
          (module): Dropout(p=0)
        )
      )
      (word_comparator): Attention(
        (alignment_networks): ModuleList(
          (0): AlignmentNetwork(
            (transform): Transform(
              (transforms): ModuleList(
                (0): Linear(in_features=300, out_features=300, bias=True)
                (1): Linear(in_features=300, out_features=300, bias=True)
              )
              (bypass_networks): ModuleList(
                (0): Bypass(
                  (highway_gate): Linear(in_features=300, out_features=300, bias=True)
                )
                (1): Bypass(


In [ ]:
hybrid_model_numericencoding.run_train(train,valid,'models/hybrid_numericencoding.pth',epochs=30,
                                      pos_neg_ration=4,batch_size=16)

## Datasets creation

In [5]:
attributes = ['Song_Name','Artist_Name','Album_Name','Genre','Price','CopyRight','Time','Released']

In [6]:
neg_dataset = dm.data.process(path='sample_data/itunes-amazon/exp4',train='all_negatives_numericencoding.csv',
                            validation='validation_numericencoding.csv',cache='exp4_neg.pth')

In [7]:
classifier = hybrid_model_numericencoding.classifier
hookF_classifier = []
hookF_classifier.append(Hook(classifier))

In [8]:
batch_size = 32
splits = MatchingIterator.splits(neg_dataset,batch_size=batch_size)

In [9]:
negative_batches = []
for batch in splits[0]:
    negative_batches.append(batch)

In [10]:
negative_classifier_inputs = []
negative_classifier_outputs = []
for batch in negative_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model_numericencoding)
    negative_classifier_inputs.append(classifier_input)
    negative_classifier_outputs.append(classifier_output)

In [11]:
negative_classifier_inputs = list(map(lambda x: x[0][0],negative_classifier_inputs))
negative_classifier_outputs = list(map(lambda x: x[0][0],negative_classifier_outputs))

In [12]:
pos_dataset = dm.data.process(path='sample_data/itunes-amazon/exp4',train='all_positives_numericencoding.csv',
                            validation='validation_numericencoding.csv',cache='exp4_pos.pth')

In [13]:
batch_size = 32
splits = MatchingIterator.splits(pos_dataset,batch_size=batch_size)

In [14]:
positive_batches = []
for batch in splits[0]:
    positive_batches.append(batch)

In [15]:
positive_classifier_inputs = []
positive_classifier_outputs = []
for batch in positive_batches:
    classifier_input,classifier_output = return_layer_input_output(hookF_classifier,batch,hybrid_model_numericencoding)
    positive_classifier_inputs.append(classifier_input)
    positive_classifier_outputs.append(classifier_output)

In [16]:
positive_classifier_inputs = list(map(lambda x: x[0][0],positive_classifier_inputs))
positive_classifier_outputs = list(map(lambda x: x[0][0],positive_classifier_outputs))

## Utility methods

In [17]:
from utilities_functions.distance_measures import find_smallest_variation_to_change

## Negative sample analysis

In [18]:
attribute_lenght= len(attributes)

current_sample = 0
#each column of this matrix is related to a specific attribute
negatives_ri_matrix = []
for batch in negative_classifier_inputs:
    for index in range(len(batch)):
        current_sample_ris = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(hybrid_model_numericencoding.classifier,batch,index,j,1)
            current_sample_ris.append(variation)
        negatives_ri_matrix.append(current_sample_ris)
        current_sample+=1

Processing sample 0 with attribute Song_Name
Processing sample 0 with attribute Artist_Name
Processing sample 0 with attribute Album_Name
Processing sample 0 with attribute Genre
Processing sample 0 with attribute Price
Processing sample 0 with attribute CopyRight
Processing sample 0 with attribute Time
Processing sample 0 with attribute Released
Processing sample 1 with attribute Song_Name
Processing sample 1 with attribute Artist_Name
Processing sample 1 with attribute Album_Name
Processing sample 1 with attribute Genre
Processing sample 1 with attribute Price
Processing sample 1 with attribute CopyRight
Processing sample 1 with attribute Time
Processing sample 1 with attribute Released
Processing sample 2 with attribute Song_Name
Processing sample 2 with attribute Artist_Name
Processing sample 2 with attribute Album_Name
Processing sample 2 with attribute Genre
Processing sample 2 with attribute Price
Processing sample 2 with attribute CopyRight
Processing sample 2 with attribute Ti

Processing sample 23 with attribute Album_Name
Processing sample 23 with attribute Genre
Processing sample 23 with attribute Price
Processing sample 23 with attribute CopyRight
Processing sample 23 with attribute Time
Processing sample 23 with attribute Released
Processing sample 24 with attribute Song_Name
Processing sample 24 with attribute Artist_Name
Processing sample 24 with attribute Album_Name
Processing sample 24 with attribute Genre
Processing sample 24 with attribute Price
Processing sample 24 with attribute CopyRight
Processing sample 24 with attribute Time
Processing sample 24 with attribute Released
Processing sample 25 with attribute Song_Name
Processing sample 25 with attribute Artist_Name
Processing sample 25 with attribute Album_Name
Processing sample 25 with attribute Genre
Processing sample 25 with attribute Price
Processing sample 25 with attribute CopyRight
Processing sample 25 with attribute Time
Processing sample 25 with attribute Released
Processing sample 26 wi

Processing sample 46 with attribute Album_Name
Processing sample 46 with attribute Genre
Processing sample 46 with attribute Price
Processing sample 46 with attribute CopyRight
Processing sample 46 with attribute Time
Processing sample 46 with attribute Released
Processing sample 47 with attribute Song_Name
Processing sample 47 with attribute Artist_Name
Processing sample 47 with attribute Album_Name
Processing sample 47 with attribute Genre
Processing sample 47 with attribute Price
Processing sample 47 with attribute CopyRight
Processing sample 47 with attribute Time
Processing sample 47 with attribute Released
Processing sample 48 with attribute Song_Name
Processing sample 48 with attribute Artist_Name
Processing sample 48 with attribute Album_Name
Processing sample 48 with attribute Genre
Processing sample 48 with attribute Price
Processing sample 48 with attribute CopyRight
Processing sample 48 with attribute Time
Processing sample 48 with attribute Released
Processing sample 49 wi

Processing sample 69 with attribute Album_Name
Processing sample 69 with attribute Genre
Processing sample 69 with attribute Price
Processing sample 69 with attribute CopyRight
Processing sample 69 with attribute Time
Processing sample 69 with attribute Released
Processing sample 70 with attribute Song_Name
Processing sample 70 with attribute Artist_Name
Processing sample 70 with attribute Album_Name
Processing sample 70 with attribute Genre
Processing sample 70 with attribute Price
Processing sample 70 with attribute CopyRight
Processing sample 70 with attribute Time
Processing sample 70 with attribute Released
Processing sample 71 with attribute Song_Name
Processing sample 71 with attribute Artist_Name
Processing sample 71 with attribute Album_Name
Processing sample 71 with attribute Genre
Processing sample 71 with attribute Price
Processing sample 71 with attribute CopyRight
Processing sample 71 with attribute Time
Processing sample 71 with attribute Released
Processing sample 72 wi

Processing sample 92 with attribute Album_Name
Processing sample 92 with attribute Genre
Processing sample 92 with attribute Price
Processing sample 92 with attribute CopyRight
Processing sample 92 with attribute Time
Processing sample 92 with attribute Released
Processing sample 93 with attribute Song_Name
Processing sample 93 with attribute Artist_Name
Processing sample 93 with attribute Album_Name
Processing sample 93 with attribute Genre
Processing sample 93 with attribute Price
Processing sample 93 with attribute CopyRight
Processing sample 93 with attribute Time
Processing sample 93 with attribute Released
Processing sample 94 with attribute Song_Name
Processing sample 94 with attribute Artist_Name
Processing sample 94 with attribute Album_Name
Processing sample 94 with attribute Genre
Processing sample 94 with attribute Price
Processing sample 94 with attribute CopyRight
Processing sample 94 with attribute Time
Processing sample 94 with attribute Released
Processing sample 95 wi

...

Processing sample 295 with attribute Song_Name
Processing sample 295 with attribute Artist_Name
Processing sample 295 with attribute Album_Name
Processing sample 295 with attribute Genre
Processing sample 295 with attribute Price
Processing sample 295 with attribute CopyRight
Processing sample 295 with attribute Time
Processing sample 295 with attribute Released
Processing sample 296 with attribute Song_Name
Processing sample 296 with attribute Artist_Name
Processing sample 296 with attribute Album_Name
Processing sample 296 with attribute Genre
Processing sample 296 with attribute Price
Processing sample 296 with attribute CopyRight
Processing sample 296 with attribute Time
Processing sample 296 with attribute Released
Processing sample 297 with attribute Song_Name
Processing sample 297 with attribute Artist_Name
Processing sample 297 with attribute Album_Name
Processing sample 297 with attribute Genre
Processing sample 297 with attribute Price
Processing sample 297 with attribute Cop

Processing sample 317 with attribute Price
Processing sample 317 with attribute CopyRight
Processing sample 317 with attribute Time
Processing sample 317 with attribute Released
Processing sample 318 with attribute Song_Name
Processing sample 318 with attribute Artist_Name
Processing sample 318 with attribute Album_Name
Processing sample 318 with attribute Genre
Processing sample 318 with attribute Price
Processing sample 318 with attribute CopyRight
Processing sample 318 with attribute Time
Processing sample 318 with attribute Released
Processing sample 319 with attribute Song_Name
Processing sample 319 with attribute Artist_Name
Processing sample 319 with attribute Album_Name
Processing sample 319 with attribute Genre
Processing sample 319 with attribute Price
Processing sample 319 with attribute CopyRight
Processing sample 319 with attribute Time
Processing sample 319 with attribute Released
Processing sample 320 with attribute Song_Name
Processing sample 320 with attribute Artist_N

Processing sample 340 with attribute Song_Name
Processing sample 340 with attribute Artist_Name
Processing sample 340 with attribute Album_Name
Processing sample 340 with attribute Genre
Processing sample 340 with attribute Price
Processing sample 340 with attribute CopyRight
Processing sample 340 with attribute Time
Processing sample 340 with attribute Released
Processing sample 341 with attribute Song_Name
Processing sample 341 with attribute Artist_Name
Processing sample 341 with attribute Album_Name
Processing sample 341 with attribute Genre
Processing sample 341 with attribute Price
Processing sample 341 with attribute CopyRight
Processing sample 341 with attribute Time
Processing sample 341 with attribute Released
Processing sample 342 with attribute Song_Name
Processing sample 342 with attribute Artist_Name
Processing sample 342 with attribute Album_Name
Processing sample 342 with attribute Genre
Processing sample 342 with attribute Price
Processing sample 342 with attribute Cop

Processing sample 362 with attribute Price
Processing sample 362 with attribute CopyRight
Processing sample 362 with attribute Time
Processing sample 362 with attribute Released
Processing sample 363 with attribute Song_Name
Processing sample 363 with attribute Artist_Name
Processing sample 363 with attribute Album_Name
Processing sample 363 with attribute Genre
Processing sample 363 with attribute Price
Processing sample 363 with attribute CopyRight
Processing sample 363 with attribute Time
Processing sample 363 with attribute Released
Processing sample 364 with attribute Song_Name
Processing sample 364 with attribute Artist_Name
Processing sample 364 with attribute Album_Name
Processing sample 364 with attribute Genre
Processing sample 364 with attribute Price
Processing sample 364 with attribute CopyRight
Processing sample 364 with attribute Time
Processing sample 364 with attribute Released
Processing sample 365 with attribute Song_Name
Processing sample 365 with attribute Artist_N

Processing sample 385 with attribute Song_Name
Processing sample 385 with attribute Artist_Name
Processing sample 385 with attribute Album_Name
Processing sample 385 with attribute Genre
Processing sample 385 with attribute Price
Processing sample 385 with attribute CopyRight
Processing sample 385 with attribute Time
Processing sample 385 with attribute Released
Processing sample 386 with attribute Song_Name
Processing sample 386 with attribute Artist_Name
Processing sample 386 with attribute Album_Name
Processing sample 386 with attribute Genre
Processing sample 386 with attribute Price
Processing sample 386 with attribute CopyRight
Processing sample 386 with attribute Time
Processing sample 386 with attribute Released
Processing sample 387 with attribute Song_Name
Processing sample 387 with attribute Artist_Name
Processing sample 387 with attribute Album_Name
Processing sample 387 with attribute Genre
Processing sample 387 with attribute Price
Processing sample 387 with attribute Cop

In [19]:
ri_norms_negative_samples = []
for ri_list in negatives_ri_matrix:
    ri_norms_negative_samples.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [20]:
import pandas as pd
variation_df = pd.DataFrame(data= ri_norms_negative_samples,columns=attributes)

In [21]:
columns = list(variation_df)
sum_ri = {}
for col in columns:
    sum_ri[col] = variation_df[col].sum()

In [22]:
sum_ri

{'Song_Name': 2987.083234190941,
 'Artist_Name': 6260.2585825920105,
 'Album_Name': 3704.05709028244,
 'Genre': 6413.990332603455,
 'Price': 3389.5156993865967,
 'CopyRight': 6076.511769771576,
 'Time': 2636.632802248001,
 'Released': 6247.0441999435425}

In [ ]:
variation_df.to_csv('experiments-results/negative_samples_ri.csv',index=False)

## Positive samples analisys

In [25]:
attribute_lenght= len(attributes)
positive_samples_ri_matrix = []
current_sample = 0

for batch in positive_classifier_inputs:
    for index in range(len(batch)):
        current_sample_ris = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(hybrid_model_numericencoding.classifier,batch,index,j,0)
            current_sample_ris.append(variation)
        positive_samples_ri_matrix.append(current_sample_ris)
        current_sample+=1

Processing sample 0 with attribute Song_Name
Processing sample 0 with attribute Artist_Name
Processing sample 0 with attribute Album_Name
Processing sample 0 with attribute Genre
Processing sample 0 with attribute Price
Processing sample 0 with attribute CopyRight
Processing sample 0 with attribute Time
Processing sample 0 with attribute Released
Processing sample 1 with attribute Song_Name
Processing sample 1 with attribute Artist_Name
Processing sample 1 with attribute Album_Name
Processing sample 1 with attribute Genre
Processing sample 1 with attribute Price
Processing sample 1 with attribute CopyRight
Processing sample 1 with attribute Time
Processing sample 1 with attribute Released
Processing sample 2 with attribute Song_Name
Processing sample 2 with attribute Artist_Name
Processing sample 2 with attribute Album_Name
Processing sample 2 with attribute Genre
Processing sample 2 with attribute Price
Processing sample 2 with attribute CopyRight
Processing sample 2 with attribute Ti

Processing sample 23 with attribute Album_Name
Processing sample 23 with attribute Genre
Processing sample 23 with attribute Price
Processing sample 23 with attribute CopyRight
Processing sample 23 with attribute Time
Processing sample 23 with attribute Released
Processing sample 24 with attribute Song_Name
Processing sample 24 with attribute Artist_Name
Processing sample 24 with attribute Album_Name
Processing sample 24 with attribute Genre
Processing sample 24 with attribute Price
Processing sample 24 with attribute CopyRight
Processing sample 24 with attribute Time
Processing sample 24 with attribute Released
Processing sample 25 with attribute Song_Name
Processing sample 25 with attribute Artist_Name
Processing sample 25 with attribute Album_Name
Processing sample 25 with attribute Genre
Processing sample 25 with attribute Price
Processing sample 25 with attribute CopyRight
Processing sample 25 with attribute Time
Processing sample 25 with attribute Released
Processing sample 26 wi

Processing sample 46 with attribute Album_Name
Processing sample 46 with attribute Genre
Processing sample 46 with attribute Price
Processing sample 46 with attribute CopyRight
Processing sample 46 with attribute Time
Processing sample 46 with attribute Released
Processing sample 47 with attribute Song_Name
Processing sample 47 with attribute Artist_Name
Processing sample 47 with attribute Album_Name
Processing sample 47 with attribute Genre
Processing sample 47 with attribute Price
Processing sample 47 with attribute CopyRight
Processing sample 47 with attribute Time
Processing sample 47 with attribute Released
Processing sample 48 with attribute Song_Name
Processing sample 48 with attribute Artist_Name
Processing sample 48 with attribute Album_Name
Processing sample 48 with attribute Genre
Processing sample 48 with attribute Price
Processing sample 48 with attribute CopyRight
Processing sample 48 with attribute Time
Processing sample 48 with attribute Released
Processing sample 49 wi

Processing sample 69 with attribute Album_Name
Processing sample 69 with attribute Genre
Processing sample 69 with attribute Price
Processing sample 69 with attribute CopyRight
Processing sample 69 with attribute Time
Processing sample 69 with attribute Released
Processing sample 70 with attribute Song_Name
Processing sample 70 with attribute Artist_Name
Processing sample 70 with attribute Album_Name
Processing sample 70 with attribute Genre
Processing sample 70 with attribute Price
Processing sample 70 with attribute CopyRight
Processing sample 70 with attribute Time
Processing sample 70 with attribute Released
Processing sample 71 with attribute Song_Name
Processing sample 71 with attribute Artist_Name
Processing sample 71 with attribute Album_Name
Processing sample 71 with attribute Genre
Processing sample 71 with attribute Price
Processing sample 71 with attribute CopyRight
Processing sample 71 with attribute Time
Processing sample 71 with attribute Released
Processing sample 72 wi

Processing sample 92 with attribute Album_Name
Processing sample 92 with attribute Genre
Processing sample 92 with attribute Price
Processing sample 92 with attribute CopyRight
Processing sample 92 with attribute Time
Processing sample 92 with attribute Released
Processing sample 93 with attribute Song_Name
Processing sample 93 with attribute Artist_Name
Processing sample 93 with attribute Album_Name
Processing sample 93 with attribute Genre
Processing sample 93 with attribute Price
Processing sample 93 with attribute CopyRight
Processing sample 93 with attribute Time
Processing sample 93 with attribute Released
Processing sample 94 with attribute Song_Name
Processing sample 94 with attribute Artist_Name
Processing sample 94 with attribute Album_Name
Processing sample 94 with attribute Genre
Processing sample 94 with attribute Price
Processing sample 94 with attribute CopyRight
Processing sample 94 with attribute Time
Processing sample 94 with attribute Released
Processing sample 95 wi

Processing sample 115 with attribute Song_Name
Processing sample 115 with attribute Artist_Name
Processing sample 115 with attribute Album_Name
Processing sample 115 with attribute Genre
Processing sample 115 with attribute Price
Processing sample 115 with attribute CopyRight
Processing sample 115 with attribute Time
Processing sample 115 with attribute Released
Processing sample 116 with attribute Song_Name
Processing sample 116 with attribute Artist_Name
Processing sample 116 with attribute Album_Name
Processing sample 116 with attribute Genre
Processing sample 116 with attribute Price
Processing sample 116 with attribute CopyRight
Processing sample 116 with attribute Time
Processing sample 116 with attribute Released
Processing sample 117 with attribute Song_Name
Processing sample 117 with attribute Artist_Name
Processing sample 117 with attribute Album_Name
Processing sample 117 with attribute Genre
Processing sample 117 with attribute Price
Processing sample 117 with attribute Cop

In [26]:
ri_norms = []
for ri_list in positive_samples_ri_matrix:
    ri_norms.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [27]:
import pandas as pd
variation_df = pd.DataFrame(data= ri_norms,columns=attributes)

In [28]:
columns = list(variation_df)
sum_ri = {}
for col in columns:
    sum_ri[col] = variation_df[col].sum()

In [29]:
variation_df.head()

,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,7.416443,13.066339,8.910498,12.662943,6.983381,13.214828,6.004185,12.095785
1,2.136887,4.404752,2.567972,4.219779,2.152426,4.470359,1.736317,4.142581
2,6.656691,13.037644,8.273711,12.339103,6.536732,13.250337,5.355540,11.901794
3,3.488452,8.492527,4.448065,7.887391,3.816852,8.639491,3.586415,8.347251
4,6.879353,12.734942,8.432244,12.403474,6.483789,12.839190,5.808014,11.691287


In [30]:
sum_ri

{'Song_Name': 814.1031942367554,
 'Artist_Name': 1280.9989819526672,
 'Album_Name': 967.4343025684357,
 'Genre': 1249.8788282871246,
 'Price': 718.3666291236877,
 'CopyRight': 1289.2608795166016,
 'Time': 695.3380416631699,
 'Released': 1186.7523791790009}

In [ ]:
variation_df.to_csv('experiments-results/positive_samples_ri_truegradient.csv',index=False,sep=';')

## Finding closer vectors

In [ ]:
from scipy.spatial.distance import euclidean
def get_closer_vector(v,batch_list):
    min_distance = 10000000
    i = 0
    best_sample = i
    for batch in batch_list:
        for sample in batch:
            curr_distance = euclidean(v.data,sample.data)
            if curr_distance< min_distance:
                min_distance = curr_distance
                best_sample = i
            i+=1
    return best_sample

In [ ]:
def get_closer_vector_on_attribute(v,batch_list,attribute_idx,attribute_lenght):
    min_distance = 1000000
    i = 0
    best_sample = i
    start_index = attribute_idx*attribute_lenght
    end_index = start_index+attribute_lenght
    v_attribute_selected = v[start_index:end_index]
    for batch in batch_list:
        for sample in batch:
            curr_distance = euclidean(v_attribute_selected.data,sample[start_index:end_index].data)
            if curr_distance < min_distance:
                min_distance = curr_distance
                best_sample = i
            i+=1
    return best_sample

In [ ]:
#test
closer_sample_index = get_closer_vector_on_attribute(positive_classifier_inputs[0][10],positive_classifier_inputs,0,150)

In [ ]:
closer_sample_index

In [ ]:
idx = get_closer_vector_on_attribute(positive_classifier_inputs[4][5],negative_classifier_inputs,0,150)

In [ ]:
idx

In [ ]:
positive_batches[4].id

In [ ]:
#lista di tuple: vettore più vicino considerando tutti gli elementi e closer solo secondo un attributo
closer_vectors = []
i = 0
for batch in negative_classifier_inputs:
    for sample in batch:
        # lista di coppie in cui il primo elemento indica il campione più vicino considerando tutti gli elementi
        # e il secondo indica il più vicino considerando il singolo attributo
        current_sample_closer_vectors = []
        for j,attribute in enumerate(attributes):
            curr_ri = negatives_ri_matrix[i][j]
            altered_sample = sample + curr_ri
            closer_index = get_closer_vector(altered_sample,positive_classifier_inputs)
            closer_on_attribute = get_closer_vector_on_attribute(altered_sample,
                                                                               positive_classifier_inputs,j,150)
            current_sample_closer_vectors.append((closer_index,closer_on_attribute))
        closer_vectors.append(current_sample_closer_vectors)
        i += 1 

In [ ]:
#lista di tuple: vettore piu' vicino, batch index e sample index
positives_closer_vectors = []
i = 0
for batch in positive_classifier_inputs:
    for sample in batch:
        current_sample_closer_vectors = []
        for j,attribute in enumerate(attributes):
            curr_ri = positive_samples_ri_matrix[i][j]
            altered_sample = sample + curr_ri
            closer_index = get_closer_vector(altered_sample,negative_classifier_inputs)
            closer_on_attribute_index = get_closer_vector_on_attribute(altered_sample,
                                                                               negative_classifier_inputs,j,150)
            current_sample_closer_vectors.append((closer_index,closer_on_attribute_index))
        positives_closer_vectors.append(current_sample_closer_vectors)
        i += 1 

In [ ]:
ordered_id_positives = []
for batch in positive_batches:
    ordered_id_positives = ordered_id_positives+batch.id

In [ ]:
ordered_id_negatives = []
for batch in negative_batches:
    ordered_id_negatives = ordered_id_negatives+batch.id

In [ ]:
closer_vectors[1]

In [ ]:
positives_closer_vectors_df = pd.DataFrame(data= positives_closer_vectors,columns =attributes)

In [ ]:
positives_closer_vectors_df.head()

In [ ]:
negatives_closer_vectors_df = pd.DataFrame(data = closer_vectors, columns = attributes)

In [ ]:
negatives_closer_vectors_df.head()

In [ ]:
positives_closer_vectors_df = positives_closer_vectors_df.applymap(lambda c: (ordered_id_negatives[c[0]],
                                                                   ordered_id_negatives[c[1]]))

In [ ]:
positives_closer_vectors_df.head()

In [ ]:
negatives_closer_vectors_df = negatives_closer_vectors_df.applymap(lambda c: (ordered_id_positives[c[0]],
                                                                   ordered_id_positives[c[1]]))

In [ ]:
negatives_closer_vectors_df.head()

In [ ]:
positives_closer_vectors_on_attribute_df = positives_closer_vectors_df.applymap(lambda c: c[1])

In [ ]:
positives_closer_vectors_on_attribute_df.head()

In [ ]:
negatives_closer_vectors_on_attribute_df = negatives_closer_vectors_df.applymap(lambda c : c[1])

In [ ]:
positives_closer_vectors_df['Pos_sample_ID']= ordered_id_positives
positives_closer_vectors_on_attribute_df['Pos_sample_ID'] = ordered_id_positives

In [ ]:
positives_closer_vectors_df.to_csv('positives_closer_vectors.csv',sep=';',index=False)

In [ ]:
positives_closer_vectors_on_attribute_df.to_csv('positives_closer_vectors_on_attribute.csv',sep=';',index=False)

In [ ]:
negatives_closer_vectors_df['Neg_sample_ID'] = ordered_id_negatives
negatives_closer_vectors_on_attribute_df['Neg_sample_ID'] = ordered_id_negatives

In [ ]:
negatives_closer_vectors_df.to_csv('negatives_closer_vectors.csv',sep=';',index=False)
negatives_closer_vectors_on_attribute_df.to_csv('negatives_closer_vector_on_attribute.csv',sep=';',index=False)

### Some tests

In [ ]:
it,variation = find_smallest_variation_to_change(hybrid_model.classifier,positive_classifier_inputs[0],20,6,0)

In [ ]:
positive_classifier_inputs[0][20].data = positive_classifier_inputs[0][20].data.copy_(positive_classifier_inputs[0][20].data+
                                                                                     variation.data)

In [ ]:
hybrid_model.classifier.forward(positive_classifier_inputs[0])[20]

In [ ]:
grads_list = []
h = positive_classifier_inputs[0].register_hook(lambda g: grads_list.append(g))

In [ ]:
out = hybrid_model.classifier.forward(positive_classifier_inputs[0])
prob = get_probabilites(out[20])

In [ ]:
from torch.autograd import Variable
true_labels = Variable (torch.cuda.FloatTensor([0,1]))